In [1]:
# Add project src to path.
import set_path
import glob

# Import modules.
import numpy as np
import time
import logging
from tqdm import tqdm

# Visulatisation
%matplotlib widget
import matplotlib.pyplot as plt

import set_path
import src.utils.ahn_utils as ahn_utils
import src.utils.las_utils as las_utils
import src.utils.plot_utils as plot_utils
import src.utils.clip_utils as clip_utils
import src.utils.math_utils as math_utils
import src.utils.bgt_utils as bgt_utils

from src.utils.interpolation import FastGridInterpolator

from pyntcloud import PyntCloud
import pandas as pd
from skimage import feature
from scipy import ndimage
import cv2 as cv
import pyransac3d as pyrsc

import open3d as o3d
from shapely.geometry import LineString
import shapely 
from scipy.optimize import curve_fit
import scipy.spatial

from multiprocessing import Pool
from sklearn.cluster import DBSCAN

from skimage.transform import probabilistic_hough_line, hough_line, hough_line_peaks

In [2]:
def plot_cloud(points, labels, plot_3d=True):

    fig = plt.figure()
    if plot_3d:
        ax = plt.axes(projection='3d')
        for label in np.unique(labels):
            mask = labels == label
            ax.scatter(points[mask][:,0],points[mask][:,1],points[mask][:,2], s=2, label=str(label))
    else:
        ax = plt.axes()
        for label in np.unique(labels):
            mask = labels == label
            ax.scatter(points[mask][:,0],points[mask][:,1], s=1,label=str(label))
    plt.legend()
    plt.show()

In [4]:
lamp_dimensions = []

for las_file in glob.iglob('../datasets/*/pointcloud/processed_*.laz'):
    print('Processing file', las_file)

    pointcloud = las_utils.read_las(las_file)
    true_labels = pointcloud.label
    points = np.vstack((pointcloud.x, pointcloud.y, pointcloud.z)).T
    labels = np.zeros(len(points))

    lamp_mask = true_labels == 14
    if lamp_mask.sum() > 4:
        clustering = (DBSCAN(eps=1, min_samples=4, p=2).fit(points[lamp_mask]))
        for cluster in np.unique(clustering.labels_):
            c_mask = clustering.labels_ == cluster
            cl_pts = points[lamp_mask][c_mask]

            cl_height = cl_pts[:,2].max() - cl_pts[:,2].min()
            min_bounding_rect, hull_points, min_dims, max_dims, center_point = math_utils.minimum_bounding_rectangle(cl_pts[:,:2])
            lamp_dimensions.append([round(min_dims,2), round(max_dims,2), round(cl_height,2)])

            #print(f'\tCluster {cluster}: {center_point} ({round(min_dims,2)}m x {round(max_dims,2)}m x {round(cl_height,2)}m)')

ld = np.array(lamp_dimensions)

Processing file ../datasets/Demo/pointcloud/processed_2386_9702.laz
Processing file ../datasets/Demo/pointcloud/processed_2397_9705.laz
Processing file ../datasets/NieuweKerkstraat/pointcloud/processed_2442_9726.laz
Processing file ../datasets/NieuweKerkstraat/pointcloud/processed_2441_9726.laz
Processing file ../datasets/Valeriusplein/pointcloud/processed_2386_9699.laz
Processing file ../datasets/Valeriusplein/pointcloud/processed_2387_9699.laz
Processing file ../datasets/Valeriusplein/pointcloud/processed_2387_9700.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2408_9765.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2408_9766.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2408_9767.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2407_9765.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2407_9766.laz
Processing file ../datasets/Kattenslootbrug/pointcloud/processed_2407_9767.laz

In [5]:
bbox = math_utils.compute_bounding_box(ld[:,1:])

fig = plt.figure()
ax = plt.axes()
ax.scatter(ld[:,1],ld[:,2], s=2, c='r')
ax.plot([bbox[0],bbox[2],bbox[2],bbox[0],bbox[0]],[bbox[1],bbox[1],bbox[3],bbox[3], bbox[1]], c='k', alpha=0.5)
ax.plot([bbox[0]-0.05,bbox[2]+0.05,bbox[2]+0.05,bbox[0]-0.05,bbox[0]-0.05],[bbox[1]-0.05,bbox[1]-0.05,bbox[3]+0.05,bbox[3]+0.05, bbox[1]-0.05], linestyle='--', c='k')
ax.set_title('Armatuur Dimensions')
ax.set_xlim((0,1))
ax.set_ylim((0,1))
ax.set_ylabel('Height (m)')
ax.set_xlabel('Width (m)')

math_utils.compute_bounding_box(ld[:,1:])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.52, 0.27, 0.85, 0.46)

In [8]:
ld.max(axis=0)

array([0.68, 0.85, 0.46])